In [ ]:
#libraries

import pandas as pd
import numpy as np
import requests
import os
from sklearn.cluster import KMeans
import folium 
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import geopandas
from unidecode import unidecode
import xlrd 
import matplotlib.pyplot as plt



In [ ]:
#request data from wikipedia website
url = "https://pt.wikipedia.org/wiki/Lista_dos_distritos_de_São_Paulo_por_população"
source = requests.get(url)

In [ ]:
#create a dataframe with Sao Paulo District Names and Population data
df = pd.read_html(source.content, header=0)[0]
df.head()

#Add HDI and Demographic Density
HDI_data_filepath = "/Users/brunonodomi/Desktop/Coursera/Sao Paulo IDH.xlsx"
df_temp2 = pd.read_excel(HDI_data_filepath, index_col=0)  
df_temp2.reset_index()
df = pd.merge(df,df_temp2, how='left', left_on='Distrito', right_on='Distritos')

#Add Real State data (Price/m2)
RealEstate_data_filepath = "/Users/brunonodomi/Desktop/Coursera/Sao Paulo m2.xlsx"
df_temp3 = pd.read_excel(RealEstate_data_filepath,index_col = 0)
df_temp3.reset_index()
df = pd.merge(df,df_temp3, how='left', left_on='Distrito', right_on='Distrito')

#Clean names data
df.Distrito = df.Distrito.str.upper()
df.Distrito = df.Distrito.apply(lambda x: unidecode(x))
df.drop(["Censo 2000"], axis =1,inplace=True)
df.rename(columns={"Densidade Demográfica (Hab/km²)": "Densidade Demográfica"}, inplace = True)
df.Distrito.replace({"SAO MIGUEL PAULISTA": "SAO MIGUEL", "b": "y"}, inplace=True)
df.dropna(0, inplace=True)

df

In [ ]:
#Check if all rows are filled
if df.isnull().values.any() == False:
    print("Data is OK")
else:
    print("There are missing data")

In [ ]:
df

In [ ]:
#Get latitude and longitude
df["Adresses"] = df["Distrito"] + ", " + "Sao Paulo, Sao Paulo"
nom = Nominatim(timeout = 5)
lat = []
lon = []

for adress in df.Adresses:
    print(adress)
    lat.append(nom.geocode(adress).latitude)
    lon.append(nom.geocode(adress).longitude)
    
df["Latitude"] = lat
df["Longitude"] = lon
df.head()

In [ ]:
#Clustering according to 10 most common venue categories

'''
I utilized the Foursquare API to explore the boroughs and segment them. 
I designed the limit as 100 venues and the radius 500 meters for each borough from their given latitude and 
longitude information. Here is the header of the result, adding venue id, venue name, 
category, latitude, and longitude information from Foursquare API.
'''

#Fousquare credentials and version
CLIENT_ID = "Y3BOULFILIOATY5RAOVGNAXQBARUD1MSZNVOQINWOL4K2AU4"
CLIENT_SECRET ="0ZDXJJYMEEQRQOZNJ1L2OUQG0HJ2RHZN255FKAWQMRKI1GT2"
VERSION = "20180604"


#Function to get venues within a certain distance (500m)
def getNearbyVenues(names, latitudes, longitudes):
    radius=2000
    LIMIT=500
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name, lat, lng)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            "4bf58dd8d48988d1ca941735")

            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
#Call function above
pizza_venues = getNearbyVenues(names=df['Distrito'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

pizza_venues.head(50)


In [ ]:
#Filtered venues specialized on pizza --> exclude bakery and bar
only_pizza_venues = pizza_venues.loc[pizza_venues["Venue Category"] == "Pizza Place"]
only_pizza_venues

In [ ]:
#Count number of pizzerias per neighborhood
count_pizza_venues = only_pizza_venues.groupby(["Neighborhood"]).count()
df = pd.merge(df, count_pizza_venues.Venue, how='left', left_on='Distrito', right_on='Neighborhood')
df.rename(columns={"Venue": "Pizzerias Density"}, inplace = True)




In [ ]:
df

In [ ]:
#Get Density of Habitants per Pizzeria
df["Habitants_Pizzeria Score"]= df["Densidade Demográfica"]/df["Pizzerias Density"]


In [ ]:
#Exclude not enough data from df
#df.drop(df[df["Pizzerias Density"] < 5].index, inplace=True)

In [ ]:
#Mapping
#Competition
state_geo = os.path.join("/Users/brunonodomi/Desktop/Coursera/distritos_sp.geojson")

# Initialize the map:
m = folium.Map(location=[-23.65, -46.65], zoom_start=10)

m.choropleth(
 geo_data=state_geo,
 name='choropleth',
 data=df,
 columns=['Distrito', 'Pizzerias Density'],
 key_on='feature.properties.nomecaps',
 fill_color='YlOrRd',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='Pizzerias Density'
)
folium.LayerControl().add_to(m)

m


In [ ]:
#import filepath as geojson file
state_geo = os.path.join("/Users/brunonodomi/Desktop/Coursera/distritos_sp.geojson")


#Populaçao
import geojson
with open(state_geo) as f:
    gj = geojson.load(f)
    
#Plot graph
import plotly.express as px

fig = px.choropleth(df, geojson = gj, color='Pizzerias Density',
                    locations="Distrito", featureidkey="properties.nomecaps",
                    projection="mercator", color_continuous_scale = "agsunset_r"
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
'''
#Population
# Initialize the map:
n = folium.Map(location=[-23.65, -46.65], zoom_start=10)

n.choropleth(
 geo_data=state_geo,
 name='choropleth',
 data=df,
 columns=['Distrito', 'População 2010'],
 key_on='feature.properties.nomecaps',
 fill_color='PuRd',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='Population'
)
folium.LayerControl().add_to(n)

n

'''

In [ ]:
#import filepath as geojson file

#Populaçao
import geojson
with open(state_geo) as f:
    gj = geojson.load(f)
    
#Plot graph
import plotly.express as px

fig = px.choropleth(df, geojson = gj, color="População 2010",
                    locations="Distrito", featureidkey="properties.nomecaps",
                    projection="mercator", color_continuous_scale = "sunsetdark"
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
#Plot graph Pop Density
import plotly.express as px

fig = px.choropleth(df, geojson = gj, color="Densidade Demográfica",
                    locations="Distrito", featureidkey="properties.nomecaps",
                    projection="mercator", color_continuous_scale = "sunsetdark"
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
'''
#IDH

# Initialize the map:
o = folium.Map(location=[-23.65, -46.65], zoom_start=10)

o.choropleth(
 geo_data=state_geo,
 name='choropleth',
 data=df,
 columns=['Distrito', 'IDH'],
 key_on='feature.properties.nomecaps',
 fill_color='YlOrRd',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='IDH'
)
folium.LayerControl().add_to(o)

#IDH
o

'''


In [ ]:
#IDH
    
#Plot graph
import plotly.express as px

fig = px.choropleth(df, geojson = gj, color="IDH",
                    locations="Distrito", featureidkey="properties.nomecaps",
                    projection="mercator", color_continuous_scale = "RdYlGn"
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
"""
#Densidade Demográfica


# Initialize the map:
p = folium.Map(location=[-23.65, -46.65], zoom_start=10)

p.choropleth(
 geo_data=state_geo,
 name='choropleth',
 data=df,
 columns=['Distrito', 'Densidade Demográfica'],
 key_on='feature.properties.nomecaps',
 fill_color='YlOrRd',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='Densidade Demográfica'
)
folium.LayerControl().add_to(p)

#Densidade Demográfica

p

"""

In [ ]:
#IDH
#Plot graph
import plotly.express as px

fig = px.choropleth(df, geojson = gj, color="IDH",
                    locations="Distrito", featureidkey="properties.nomecaps",
                    projection="mercator", color_continuous_scale = "agsunset"
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show

In [ ]:
'''
#Habitants_Pizzeria Scor#Habitants_Pizzeria Scor'e

# Initialize the map:
q = folium.Map(location=[-23.65, -46.65], zoom_start=10)

q.choropleth(
 geo_data=state_geo,
 name='choropleth',
 data=df,
 columns=['Distrito', 'Habitants_Pizzeria Score'],
 key_on='feature.properties.nomecaps',
 fill_color='YlOrRd',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='Pizzeria-Habitants Ratio'
)
folium.LayerControl().add_to(q)

q
'''


In [ ]:
#Habitants_Pizzeria Score
    
#Plot graph
import plotly.express as px

fig = px.choropleth(df, geojson = gj, color="Habitants_Pizzeria Score",
                    locations="Distrito", featureidkey="properties.nomecaps",
                    projection="mercator", color_continuous_scale = "plasma"
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
'''
#Preço m2

# Initialize the map:
r = folium.Map(location=[-23.65, -46.65], zoom_start=10)

r.choropleth(
 geo_data=state_geo,
 name='choropleth',
 data=df,
 columns=['Distrito', 'Preço m2'],
 key_on='feature.properties.nomecaps',
 fill_color='YlOrRd',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='Preço m2'
)
folium.LayerControl().add_to(r)

r
'''



In [ ]:
#import filepath as geojson file
import geojson
with open(state_geo) as f:
    gj = geojson.load(f)
    
#Plot graph
import plotly.express as px

fig = px.choropleth(df, geojson = gj, color="Preço m2",
                    locations="Distrito", featureidkey="properties.nomecaps",
                    projection="mercator", color_continuous_scale = "ylorRd"
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()



In [ ]:
##Clustering
#columns_to_cluster = ["IDH","Densidade Demográfica"]
df2 = df[["Distrito","IDH","Preço m2","Habitants_Pizzeria Score"]]

In [ ]:
df2 = df2.fillna(0)

In [ ]:
df2

In [ ]:
#Normalize over MinmaxScaler to get 0 to 1 values (instead of 0 mean of StandardScaler())
from sklearn.preprocessing import MinMaxScaler
#from sklearn.preprocessing import StandardScaler

X = df2.values[:,1:]
Clus_dataSet = MinMaxScaler().fit_transform(X)

In [ ]:
from sklearn.cluster import KMeans

sum_of_squared_distances =[]
K = range(1,15)
for k in K:
    kmeans = KMeans(n_clusters=k).fit(Clus_dataSet)
    sum_of_squared_distances.append(kmeans.inertia_)
    

In [ ]:
plt.plot(K,sum_of_squared_distances,"bo-")
plt.xlabel("k")
plt.ylabel("Sum of squared distances")
plt.title("Elbow method for Optimal K")

In [ ]:
#Modeling
clusterNum = 5
k_means = KMeans(init = "k-means++", n_clusters = clusterNum, n_init = 12)
k_means.fit(X)
labels = k_means.labels_
print(labels)

In [ ]:
df["Clus_km"] = labels
df.head(5)

In [ ]:
df4 = pd.DataFrame(X, columns=["IDH","Preço m2","Habitants_Pizzeria Score"]) 
df4["labels"]=df.Clus_km
df4["Distrito"]=df.Distrito
df4["labels"].apply(lambda x:str(x))
df4.fillna(0,inplace =True)


In [ ]:
df4

In [ ]:
#transform object type to float type, so it can be used to plot as size parameter
df4.loc[:,'IDH'] = df["IDH"].astype(np.float)
df4.info()

In [ ]:
df4

In [ ]:
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

cmap = sns.cubehelix_palette(dark=.3, light=.8, as_cmap=True)


HB_Ratio = df4["Habitants_Pizzeria Score"]
plt.figure(figsize=(15,8))
ax = sns.scatterplot(x="Preço m2", y="Habitants_Pizzeria Score",
                     hue="labels",size = HB_Ratio,
                     palette="Set2",
                     data = df4)
ax.plot()

In [ ]:
fig = px.scatter(df4, x="Preço m2", y="IDH",
             size="Habitants_Pizzeria Score", color="labels",
             hover_name="Distrito",trendline="ols",color_continuous_scale ="Portland",
             log_x=True, size_max=60, width=1000, height=600)

fig.show()

In [ ]:
#import filepath as geojson file
import geojson
with open(state_geo) as f:
    gj = geojson.load(f)
    
#Plot graph
import plotly.express as px

fig = px.choropleth(df4, geojson = gj, color="labels",
                    locations="Distrito", featureidkey="properties.nomecaps",
                    projection="mercator", color_continuous_scale ="Portland"
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
                title_text='São Paulo')
fig.show()


In [ ]:
df_fig1 = df
df_fig1.rename(columns={'Distrito':'District',
                          'População 2010':'Population',
                          'IDH':'HDI',
                          'Densidade Demográfica': 'Pop_Density',
                          'Regiao':'Zone',
                          'Preço m2':'Price per m2'
                       }, 
                 inplace=True)
df_fig1.head()

In [ ]:
df_fig1.iloc[:, 2:7]
